In [17]:
import csv
from getpass import getpass
from msedge.selenium_tools import Edge,EdgeOptions
from tqdm import tqdm
from time import sleep
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

def get_tweet_data(card):
    username = card.find_element_by_xpath('.//span').text
    handle = card.find_element_by_xpath('.//span[contains(text(),"@")]').text
    try:
        postdate = card.find_element_by_xpath('.//time').get_attribute('datetime')
    except NoSuchElementException:
        return
    content = card.find_element_by_xpath('.//div[2]/div[2]/div[2]').text
    reply_cnt = card.find_element_by_xpath('.//div[@data-testid="reply"]').text
    retweet_cnt = card.find_element_by_xpath('.//div[@data-testid="retweet"]').text
    like_cnt = card.find_element_by_xpath('.//div[@data-testid="like"]').text

    tweet = (username, handle, postdate, content, reply_cnt, retweet_cnt, like_cnt)
    return tweet

def get_tweet(tweet_data, tweet_ids, driver):
    scrolling = True
    tweet_count = 0
    last_position = driver.execute_script('return window.pageYOffset')
    
    while scrolling and tweet_count < 500:
        #Pegar tweets
        cards = driver.find_elements_by_xpath('//article[@data-testid="tweet"]')
        for card in cards[-15:]:
            try:
                data = get_tweet_data(card)
                if data:
                    tweet_id = ''.join(data)
                    if tweet_id not in tweet_ids:
                        tweet_ids.add(tweet_id)
                        tweet_data.append(data)
                        tweet_count += 1
            except:
                pass

        scroll_attemp = 0
        while True:
            driver.execute_script('window.scrollTo(0,document.body.scrollHeight)')
            sleep(1)

            curr_position = driver.execute_script('return window.pageYOffset')
            if curr_position == last_position:
                scroll_attemp += 1

                if scroll_attemp >= 10:
                    reason_to_stop = 'scrooling limit reached'
                    scrolling = False
                    break
                else:
                    sleep(5)
            else:
                last_position = curr_position
                break
       

options = EdgeOptions()
options.use_chromium = True
driver = Edge(options=options)

#Navegar na tela de login
driver.get('https://twitter.com/i/flow/login')

#Espera implícita 
wait = WebDriverWait(driver, 60)
wait.until(EC.visibility_of_element_located((By.XPATH, '//input[@name="text"]')))

#Passar nome de usuário
username = driver.find_element_by_xpath('//input[@name="text"]')
username.send_keys('SousaC20')
username.send_keys(Keys.RETURN)

wait.until(EC.visibility_of_element_located((By.XPATH, '//input[@name="password"]')))

#Passar senha
my_password = getpass()
password = driver.find_element_by_xpath('//input[@name="password"]')
password.send_keys(my_password)
password.send_keys(Keys.RETURN)


wait = WebDriverWait(driver, 60)
wait.until(EC.visibility_of_element_located((By.XPATH, '//input[@aria-label="Consulta de busca"]')))

#Pesquisar
search_input = driver.find_element_by_xpath('//input[@aria-label="Consulta de busca"]')
search_input.send_keys('#Rouanet')
search_input.send_keys(Keys.RETURN)

wait.until(EC.visibility_of_element_located((By.LINK_TEXT, 'Mais recentes')))

#Pegar os tweets mais recents
driver.find_element_by_link_text('Mais recentes').click()

#Interar sobre cards e extrair os dados
wait.until(EC.visibility_of_element_located((By.XPATH, '//article[@data-testid="tweet"]')))

tweet_data = []
tweet_ids = set()

try:
    for i in tqdm(range(10)):
        get_tweet(tweet_data, tweet_ids, driver)
        sleep(4)
        
        
except:
    pass

    
with open('twitter_scraping_result2.csv', 'w', newline = '', encoding = 'utf-8') as f:
    header = ['Username', 'Handle', 'Timestamp', 'Text', 'Replys', 'Retweets', 'Likes']
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(tweet_data)



········


 60%|████████████████████████████████████████████████                                | 6/10 [1:06:40<44:27, 666.78s/it]


In [19]:
try:
    for i in tqdm(range(10)):
        get_tweet(tweet_data, tweet_ids, driver)
        sleep(4)
        
        
except:
    pass

    
with open('twitter_scraping_result2.csv', 'w', newline = '', encoding = 'utf-8') as f:
    header = ['Username', 'Handle', 'Timestamp', 'Text', 'Replys', 'Retweets', 'Likes']
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(tweet_data)


 60%|█████████████████████████████████████████████████▏                                | 6/10 [29:29<19:39, 294.83s/it]


In [16]:
tweet_count

1923